In [38]:
# 1.créer un nouvel env conda à partir du terminal
# conda create --name asr_rag python=3.10

# 2. activer cet environnement
# conda activate asr_rag

# 3. installer les dépendences
# pip install -r requirements.txt

# 4. installer ollama: https://www.ollama.com/download


# 5. installer le modème d'OllamaEmbeddings
# ollama pull embeddinggemma

# 6. créer une clé api sur openrouter pour utiliser des llm gratuitement

# 7. mettre votre clé dans le fichier .env (racine du repertoire courant)

In [33]:
from openai import OpenAI, AsyncOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain.schema.document import Document
from langchain_unstructured import UnstructuredLoader
from pathrag_retriever import create_graphdb, load_existing_graphdb, load_knowledgeGraph_vis
import time
import os
from dotenv import load_dotenv

load_dotenv(".env")

OPENROUTER_API_KEY=os.getenv("OPENROUTER_API_KEY")


#### Création ou chargement d'un graphe existant:
> Lorsque vous executer la cellule ci dessous, vous avez 2 options proposées:
> 1. Créer un nouveau graphe
> 2. Charger un graphe existant
> 
> Un prompt de sélection s'affichera en haut du notebook<br>
> Saisir l'action désirée dans le , suivre instructions

> Si vous voulez charger un graphe déjà crée et que vous ne savez plus son nom, retrouver le dans le fichier `graphrag_hashes.json` (dans la racine du dossier), attribut `Nom du doc`

> Si vous voulez modifier le LLM utilisé pour la création du graphe ou sa lecture, allez dans `pathrag_retriever.py`, sous la ligne 36( Choix du LLM OpenRouter), et prenez un modèle valide sur openrouter (attention à prendre un modèle qui supporte les `structured_outputs`, sur le site openrouter, à filtrer sur le paneau à gauche dans la liste des `Supported parameters`)

#### 1. Créer un graphe, ou charger en un déjà crée

In [ ]:
# 1.
import nest_asyncio

nest_asyncio.apply()

# remetre à plat le text
filename="audio-text.txt"
loader = UnstructuredLoader(filename)

txt_docs = loader.load()
text=""
for doc in txt_docs:
    text+=doc.page_content


r=input("Saisir 'C' pour créer un nouveau graphe, 'L' pour charger un graphe existant")

# créer un nouveau graphe
messages=None
if r=='C':
    doc_name_graph=input('Saisir un nom unique pour votre graphe')
    print(f"Le nom de votre graphe est {doc_name_graph}")
    messages= create_graphdb(
        text=text, 
        doc_name=doc_name_graph, # il faut donner un nom unique permettant d'identifier et charger le graph les prochaines fois
    )
# charger un graphe existant
elif r=='L':
    doc_name_graph=input('Saisir le nom du graphe à charger')
    print(f"Le nom de votre graphe est {doc_name_graph}")

    messages=load_existing_graphdb(doc_name_graph)
else:
    print('Option invalide')



if messages:
    pipeline_args={}
    for feedback in messages:
        if isinstance(feedback, str):
            print(feedback)
        elif isinstance(feedback, dict):
            pipeline_args[f"graphrag_pipeline_{doc_name_graph}"]=feedback["pipeline_args"]
            


INFO:PathRAG:Logger initialized for working directory: /home/chougar/Téléchargements/RAG/RAG/storage/graph_stores/da824d7bce695f72eb26e2aceeedd4a69a07419e461e226546f40c47678a79a2
INFO:PathRAG:Load KV llm_response_cache with 0 data
INFO:PathRAG:Load KV full_docs with 1 data
INFO:PathRAG:Load KV text_chunks with 3 data
INFO:PathRAG:Loaded graph from /home/chougar/Téléchargements/RAG/RAG/storage/graph_stores/da824d7bce695f72eb26e2aceeedd4a69a07419e461e226546f40c47678a79a2/graph_chunk_entity_relation.graphml with 42 nodes, 36 edges
INFO:nano-vectordb:Load (42, 768) data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': '/home/chougar/Téléchargements/RAG/RAG/storage/graph_stores/da824d7bce695f72eb26e2aceeedd4a69a07419e461e226546f40c47678a79a2/vdb_entities.json'} 42 data
INFO:nano-vectordb:Load (36, 768) data
INFO:nano-vectordb:Init {'embedding_dim': 768, 'metric': 'cosine', 'storage_file': '/home/chougar/Téléchargements/RAG/RAG/storage/graph_stores/da824d7bc

Le nom de votre graphe est IA seconde conscience

        ----------------
        #### Graph RAG retriever
        Chargement de la base Graph RAG
    
**✅ Graph RAG chargé**


#### 2. Poser vos questions au graphrag

In [36]:
from PathRAG import QueryParam
import asyncio

def stream_pathRAG_response(stream_resp):
    async def stream_response():        
        # Process the async generator
        async for chunk in stream_resp:
            print(chunk or "", end="")


    # Run in Streamlit's existing event loop
    loop = asyncio.get_event_loop()
    loop.run_until_complete(stream_response())

# question="résume ce texte dans sa langue source"
question = "Quels sont les principaux thèmes de ce texte et les questions qui peuvent être posées ?"

resp=pipeline_args[f"graphrag_pipeline_{doc_name_graph}"]["rag"].query(query= question, param=QueryParam(mode="hybrid", stream=True))

stream_pathRAG_response(resp)

INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:PathRAG:kw_prompt result:
INFO:PathRAG:{
  "high_level_keywords": ["Principaux thèmes", "Questions à poser", "Analyse de texte"],
  "low_level_keywords": ["Thèmes clés", "Interrogations", "Compréhension", "Contenu"]
}
INFO:PathRAG:Local query uses 24 entites, 15 relations, 3 text units
INFO:PathRAG:Global query uses 34 entites, 32 relations, 3 text units
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


response all ready
Les principaux thèmes abordés dans ce texte tournent autour de l'intelligence artificielle (IA), de la créativité humaine, et des implications sociétales et psychologiques de l'IA. Voici une analyse des thèmes centraux et des questions qu'ils soulèvent.

---

### **1. Créativité humaine vs IA**  
**Thèmes :**  
- Le texte distingue la créativité humaine (métaphores, originalité, motivation artistique) de l'optimisation algorithmique de l'IA (production de texte basée sur des contraintes et des données existantes).  
- L'exemple du concours littéraire entre **Hervé Le Tellier** (Goncourt 2020) et **ChatGPT** illustre cette opposition.  

**Questions possibles :**  
- L'IA peut-elle vraiment être "créative", ou ne fait-elle que reproduire des schémas existants ?  
- Dans quelle mesure la créativité humaine est-elle mécanique elle-même (habitudes, réflexes) ?  

---

### **2. Impacts professionnels de l'IA**  
**Thèmes :**  
- La crainte du remplacement des métiers créa

============================
### RAG vectoriel
1. Embedding du document -> renseigner le nom de votre fichier dans `filename` et le nom de votre DB dans `doc_name_hybrid`
2. Setup du retriever / reranker / llm

In [28]:
#1
from langchain.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_community.retrievers import TFIDFRetriever
from langchain_unstructured import UnstructuredLoader

# chargement et fragmentation du doc
## Nom du doc à traiter
filename="audio-text.txt"

## Nom pour la base vectorielle
doc_name_hybrid="L-IA-notre-deuxieme-conscience_sample" # nom de doc significatif



#========= choix du modèle d'embedding
"""
    Le modèle choisi impacte la qualité du retriever, mais aussi le temps de traitement
    Si le déploiement est prévu sur une VM limitée, un modèle plus petit est nécessaire
    Explorer les comparatifs: https://huggingface.co/spaces/mteb/leaderboard

"""
# Utiliser OllamaEmbeddings avec le modèle local "embeddinggemma"
embeddings = OllamaEmbeddings(model="embeddinggemma")


# loader = UnstructuredFileLoader(filename)
loader = UnstructuredLoader(filename)

txt_doc = loader.load()
print(f"Loaded {len(txt_doc)} documents from {filename}")


#======== choix des paramètres de fragmentation
"""
    la taille du chunck_size est très important dans l'accès à une info précise
    une plus petite taille permet de cibler de courts passages contenant l'info nécessaire à des réponses précises:
        * lieu du projet
        * dates du projet
        * budget ...    
    l'envoi de passages plus courts au llm évite une dispertion de son attention
"""

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

docs = text_splitter.split_documents(txt_doc)

# Filter out complex metadata (e.g., lists, dicts)
docs = [Document(doc.page_content) for doc in docs]


# Conversion des docs en embeddings 
chroma_db = Chroma.from_documents(
    docs,
    embedding=embeddings,
    persist_directory=f'./storage/vector_scores/{doc_name_hybrid.replace(" ","_")}',
    collection_name=doc_name_hybrid.replace(" ","_")
)

retriever=chroma_db.as_retriever()





# ...existing code...
all_docs = chroma_db.get()
print("Nb de chuncks:", len(all_docs['documents']))  # This will print the total number of docs stored
# ...existing code...

Loaded 1 documents from audio-text.txt


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


Nb de chuncks: 51


In [29]:
for c in all_docs["documents"][:4]:
    print(c, "\n=============")

[Nathan Devert]: France Culture. [Nathan Devert]: France Culture, sans préjuger. Nathan Devert. [Nathan Devert]: Comment expliquer les progrès phénoménaux que semble avoir accompli l'intelligence artificielle au cours de ces dernières années ? Depuis l'apparition de ChatGPT en novembre 2022, rapidement suivi par d'autres agents conversationnels, cette révolution technologique aux multiples aspects, paraît désormais capable d'exécuter de nombreuses tâches intellectuelles sur lesquelles l'esprit humain pensait jusqu'alors exercer un monopole. Écrire des articles, synthétiser des documents, traiter des données dans n'importe quel domaine, diagnostiquer une maladie, rédiger une dissertation, ou pourquoi pas, un scénario de film. Non contente de révolutionner le monde du travail, ses prouesses stupéfiantes de la technique soulèvent une interrogation majeure dans le domaine de la philosophie de l'esprit. Faut-il en déduire que l'intelligence, n'étant pas le propre d'un cerveau biologique et 

In [ ]:
#2
from langchain.vectorstores import Chroma
from langchain.embeddings import OllamaEmbeddings
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers import TFIDFRetriever
from langchain.schema.document import Document
from openai import OpenAI, AsyncOpenAI
import asyncio
import json
import re
import os


class RAG_hybrid():
    def __init__(self, model):
        self.model=model
        self.retrieved_docs=[]
        self.semantic_retriever_topK=10
        self.sparse_retriever_topK=10
        self.history=[]
        self.llm_client = AsyncOpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=OPENROUTER_API_KEY,
        )
        self.reranker_llm="mistralai/mistral-small-3.1-24b-instruct"
        self.doc_name_hybrid=doc_name_hybrid
        self.reranker_score_thresh=5
        self.reranked_doc=[]

    def semanticRetriever(self):
        # 1. Semantic Retriever (Chroma + OllamaEmbeddings)
        embeddings = OllamaEmbeddings(model="embeddinggemma")
        if self.doc_name_hybrid == 'None':
            return "Error: fournir le nom du document"
        
        chroma_db = Chroma(
            persist_directory=f'./storage/vector_scores/{self.doc_name_hybrid.replace(" ","_")}',
            collection_name=self.doc_name_hybrid.replace(" ","_"),
            embedding_function=embeddings
        )

        semantic_retriever=chroma_db.as_retriever(search_type="mmr", k=self.semantic_retriever_topK)

        self.chroma_db=chroma_db
        self.semantic_retriever=semantic_retriever

        return "Success: ChromaDB setup avec succes"
    
    def sparseRetriever(self):
        # 2. Sparse Retriever (TF-IDF)

        # Récupérer TOUS les documents depuis Chroma
        all_data = self.chroma_db.get(include=["documents", "metadatas"])

        # Convertir en liste de `Document` objects pour LangChain
        docs = [
            Document(page_content=text, metadata=meta or {})  # <-- Si meta est None, on met {}
            for text, meta in zip(all_data["documents"], all_data["metadatas"])
        ]

        # Créer le retriever TF-IDF
        sparse_retriever = TFIDFRetriever.from_documents(
            documents=docs,
            k=self.sparse_retriever_topK,
            tfidf_params={"min_df": 1, "ngram_range": (1, 2)}
        )

        self.sparse_retriever= sparse_retriever
    
    def ensembleRetriever(self):
        # 3. Ensemble Retriever (Semantic + Sparse)
        ensemble_retriever = EnsembleRetriever(
            retrievers=[self.semantic_retriever, self.sparse_retriever],
            weights=[0.5, 0.5]
        )

        self.ensemble_retriever=ensemble_retriever

    async def reranker(self, results, query):


        async def llm_eval(doc, query):
            system_prompt="""
                You're an expert assistant in reranking documents against a question.
                Your role is to compare the question with a document and give a score from 0 to 10, where:
                0=document out of context, unable to answer the question
                10=highly relevant document, able to answer the question
                                
                The expected final output is the score in json format
                Example:
                ```json{"score": 5}```
                
                Always end your answer with this format                
            """            
            response = await self.llm_client.chat.completions.create(
                model=self.reranker_llm,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": f"La question est: {query}\n Le document à évaluer est le suivant\n: {doc}" }
                ],
                temperature=0,
            )
            # Post-process to extract only the JSON part if extra text is present
            content = response.choices[0].message.content
            # Try to extract the JSON block if the model adds extra text
            match = re.search(r"\{.*?\}", content, re.DOTALL)
            if match:
                content = match.group(0)

            # extract score
            score=None
            try:
                score=content.replace("```json", "").replace("```", "")
                
                score= json.loads(score)
                score=score["score"]
            except Exception as e:
                print(e)                
            
            return {"content": doc, "score": score}


        tasks=[llm_eval(doc.page_content, query) for doc in results]
        scored_docs= await asyncio.gather(*tasks)
        i=1

        for doc in scored_docs:
          
            print(f'chunk {i} score: {doc["score"]}')
            i+=1

        filtred_docs=[d for d in scored_docs if int(d["score"])>=self.reranker_score_thresh]
        # print(f"scored docs; \n{scored_docs}")
        self.reranked_doc=filtred_docs

        return filtred_docs

    async def ask_llm(self, query):
        # 5. Final processing step with an LLM (e.g., OpenAI via OpenRouter)

        # init retrievers
        status=self.semanticRetriever()
        if "Error" in status:
            return status
        
        self.sparseRetriever()
        self.ensembleRetriever()

        # retrieve relevant docs
        results = self.ensemble_retriever.get_relevant_documents(query)
        print(f"Nb of retrieved docs: {len(results)}")

        # rerank
        scored_results=await self.reranker(results, query)
        
        # Concatenate retrieved documents for context
        context = "\n".join([f"Fragment: \n{doc['content']}\n" for doc in scored_results])

        print(f"Context lenght: {len(context.split(' '))} words")
        llm_prompt = f"""
            Answer the question based **only** on the provided context.  

            - If the context contains enough information to provide a complete or partial answer, use it to formulate a detailed and factual response.  
            - If the context lacks relevant information, respond with: "I don't know."  

            ### **Context:**  
            {context}  

            ### **Question:**  
            {query}  

            ### **Answer:**  
            Provide a clear, factual, and well-structured response based on the available context. Avoid speculation or adding external knowledge.  
        """

        llm_completion = await self.llm_client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are an expert in document Q/A and document synthesis"},
                {"role": "user", "content": llm_prompt}
            ],
            temperature=0.2,
            stream=True
        )

        final_answer = ""
        print("Réponse:\n=========")
        async for chunk in llm_completion:
            if hasattr(chunk.choices[0].delta, "content") and chunk.choices[0].delta.content:
                final_answer += chunk.choices[0].delta.content
                print(chunk.choices[0].delta.content, end="", flush=True)
        
        self.history+=[
            {"role": "user", 'content': query},
            {"role": "assistant", "content": final_answer}
        ]
        
        return final_answer



In [32]:
rag_hybrid=RAG_hybrid(model="mistralai/mistral-small-3.2-24b-instruct")

# 4. Ask a question
question = "Quels sont les principaux thèmes de ce texte et les questions qui peuvent être posées ?"
results = await rag_hybrid.ask_llm(question)

Nb of retrieved docs: 8


INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


chunk 1 score: 4
chunk 2 score: 9
chunk 3 score: 4
chunk 4 score: 9
chunk 5 score: 2
chunk 6 score: 3
chunk 7 score: 2
chunk 8 score: 7
Context lenght: 504 words


INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


Réponse:
### **Answer:**

#### **Principaux thèmes du texte:**

1. **Créativité et Intelligence Artificielle (IA):**
   - Le texte explore la capacité de l'IA à créer des métaphores et des analogies, en soulignant que cela relève davantage de l'optimisation mathématique que de la véritable créativité.
   - Il est mentionné que l'humain apporte une touche unique de créativité, de vision métaphorique et d'inconscient, qui trouble et enrichit les productions de l'IA.

2. **Philosophie de l'esprit et Intelligence:**
   - Le texte aborde la question de savoir si l'intelligence peut être implémentée dans des machines, et si l'IA pourrait un jour remplacer la conscience humaine.
   - Il pose également la question inverse : comment la machine peut-elle imiter les œuvres de l'esprit humain, et ce que cela révèle sur la mécanisabilité de la vie mentale.

3. **Mécanique et Créativité Humaine:**
   - Il est suggéré que la créativité humaine pourrait être vue comme une addition de méthodes, d'habit